# Basic Functions used

## Pool BN

A pooling followed by a batch normalization, for both maxpooling and average pooling methods.

In [ ]:
import tensorflow.keras as K

class PoolBN(K.layers.Layer):
  """
  Description
  ---------------
  This class is used to handle both the max pooling layer and the average pooling layer.
  In both cases, the pooling is followed by a layer of batch normalization.
  """
  def __init__(self, pool_type, C, kernel_size, stride, padding):
    """
    Description
    ---------------
    Creates an instance of the class PoolBN

    Input(s)
    ---------------
    pool_type: string (either "max" or "avg")
    C: int, axis on which to perform batch normalization
    kernel_size: int or enumerable of ints
    stride: int or enumerable of ints
    padding: int

    Output(s)
    ---------------
    No output
    """
    super().__init__()
    self.pool_type = pool_type
    self.channels = C
    self.kernel_size = kernel_size
    self.stride = stride
    self.padding = padding
    # self.architecture = K.models.Sequential()
    if pool_type == "max":
      # self.architecture.add(K.layers.MaxPooling2D(kernel_size, stride, padding))
      self.pool = K.layers.MaxPooling2D(kernel_size, stride, padding)
    elif pool_type == "avg":
      # self.architecture.add(K.layers.AveragePooling2D(kernel_size, stride, padding))
      self.pool = K.layers.AveragePooling2D(kernel_size, stride, padding)
    else:
      raise ValueError()
    self.BN = K.layers.BatchNormalization()
  
  def get_config(self):
        config = super().get_config()
        config.update({
            "pool_type": self.pool_type,
            "C": self.channels,
            "kernel_size": self.kernel_size,
            "stride": self.stride,
            "padding": self.padding,
        })
        return config
  
  def call(self, x):
    """
    Description:
    ---------------
    Similar to the forward function in tensorflow, this function is used to train the AI.

    Input(s)
    ---------------
    x : tf.Tensor

    Output(s)
    ---------------
    output: tf.Tensor
    """
    # print('pooling',x)
    out = self.pool(x)
    out = self.BN(out)
    return out


Test of the pool BN method

In [ ]:
import tensorflow as tf
test_tensors = tf.ones((3,3,3,3))
test_tensors_2 = tf.ones((3,3,3,3))
test_model = K.models.Sequential()
# test_pool = PoolBN("max", 3, 3, 1, "same")
test_model.add(PoolBN("max", 3, 3, 1, "same"))
test_model.add(PoolBN("avg", 3, 3, 1, "same"))
# test_model.add(SepConv(3,3,3,1,'valid'))

# test_model.add(Layer()) # this is apparently a identity layer
test_model.compile(loss = "mse")
test_model.fit(test_tensors, tf.ones(3,3))
test_model.summary()

1/1 [==============================] - 2s 2s/step - loss: 1.0000
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 pool_bn (PoolBN)            (None, 3, 3, 3)           12        
                                                                 
 pool_bn_1 (PoolBN)          (None, 3, 3, 3)           12        
                                                                 
Total params: 24
Trainable params: 12
Non-trainable params: 12
_________________________________________________________________


## Drop path

Changes to probability of keeping the path in later iterations (not used here, except for experimentations)

In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp

class drop_path(K.layers.Layer):
  def __init__(self, p=0):
    """
    Description
    ---------------
    Creates an instance of the class drop_path.

    Input(s)
    ---------------
    p: float (0 if you don't want, else, between 0 and 1)

    Output(s)
    ---------------
    No output
    """
    super().__init__()
    self.p=p
  
  def get_config(self):
        config = super().get_config()
        config.update({
            "p": self.p,
        })
        return config
  
  def call(self, x):
    """
    Description:
    ---------------
    Similar to the forward function in tensorflow, this function is used to train the AI.

    Input(s)
    ---------------
    x : tf.Tensor

    Output(s)
    ---------------
    output: tf.Tensor
    """
    if self._trainable and self.p>0:
      keep_prob = 1. - self.p
      shape = (tf.shape(x)[0],) + (1,) * (len(tf.shape(x)) - 1)
      mask = K.backend.random_bernoulli(shape, keep_prob, dtype = tf.float32)
      return x/keep_prob*mask
    return x


In [ ]:
import tensorflow as tf
nb_input = 10
test_tensors = tf.ones((nb_input,3,3,3))
test_model = K.models.Sequential()
test_model.add(PoolBN("max", 3, 3, 1, "same"))
test_model.add(PoolBN("max", 3, 3, 2, "same"))
test_model.add(drop_path(0.3))

test_model.add(tf.keras.layers.Layer()) # this is an identity layer
test_model.compile(loss = "mse")
test_model.fit(test_tensors, tf.ones((nb_input,2,2,3)))
test_model.summary()

1/1 [==============================] - 1s 613ms/step - loss: 1.0000
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 pool_bn_2 (PoolBN)          (None, 3, 3, 3)           12        
                                                                 
 pool_bn_3 (PoolBN)          (None, 2, 2, 3)           12        
                                                                 
 drop_path (drop_path)       (None, 2, 2, 3)           0         
                                                                 
 layer (Layer)               (None, 2, 2, 3)           0         
                                                                 
Total params: 24
Trainable params: 12
Non-trainable params: 12
_________________________________________________________________


## Factorized reduce

Reduce the size of the model using factorized pointwise convolution (replaces identity in reduction cells)

In [ ]:
import tensorflow as tf
class FactorizedReduce(tf.keras.layers.Layer):
  """
  Reduce feature map size by factorized pointwise (stride=2).
  """
  def __init__(self, C_in, C_out, affine=True):
    """
    Description
    ---------------
    This function initializes an instance of the FactorizedReduce class.
    Input(s)
    ---------------
    C_in: int
    C_out: int
    Output(s)
    ---------------
    No output
    """
    self.c_in = C_in
    self.c_out = C_out
    self.affine = affine
    super().__init__()
    self.relu = tf.keras.layers.ReLU()
    self.conv2d_1 = tf.keras.layers.Conv2D(filters=C_out // 2 + C_out % 2,
                                      strides=2,
                                      padding='same' ,
                                      data_format="channels_last",
                                      kernel_size=3,
                                      use_bias=False)
    self.conv2d_2 = tf.keras.layers.Conv2D(filters=C_out // 2,
                                      strides=2,
                                      padding='same' ,
                                      data_format="channels_last",
                                      kernel_size=3,
                                      use_bias=False)
    self.concatenation = tf.keras.layers.Concatenate()
    self.batch_normalisation = tf.keras.layers.BatchNormalization()
  
  def get_config(self):
        config = super().get_config()
        config.update({
            "C_in": self.c_in,
            "C_out": self.c_out,
            "affine": self.affine,
        })
        return config

  def call(self, x):
    """
    Description
    ---------------
    This function initializes an instance of the FactorizedReduce class.
    Input(s)
    ---------------
    C_in: int
    C_out: int
    Output(s)
    ---------------
    No output
    """
    x2 = tf.keras.layers.ReLU()(x)
    # print("factorized reduce x2", x2)
    # print("factorized reduce x2.shape", x2.shape)
    out_1 = self.conv2d_1(x2)
    out_2 = self.conv2d_2(x2[:,1:,1:,:])
    # print("out_1", out_1)
    # print("out_2", out_2)
    out = self.concatenation([out_1,out_2])
    out = self.batch_normalisation(out)
    return out

In [ ]:
import tensorflow as tf
nb_input = 10
shape_1 = 100
shape_2 = 100
## channels_inside = 10, channels input = 40
channels = 20
input_channels = 80
print("input_channels :", input_channels, "\nchannels :", channels)
test_tensors = tf.ones((nb_input,shape_1,shape_2,input_channels))
test_model = K.models.Sequential()
# test_pool = PoolBN("max", 3, 3, 1, "same")
test_model.add(PoolBN("max", input_channels, channels, 1, "same"))
test_model.add(PoolBN("max", channels, channels, 2, "same"))
test_model.add(FactorizedReduce(channels,channels))
# test_model.add(drop_path(0.3))
# test_model.add(SepConv(3,3,3,1,'valid'))

# test_model.add(Layer()) # this is apparently a identity layer
test_model.compile(loss = "mse")
test_model.fit(test_tensors, tf.ones((nb_input,shape_1//4,shape_2//4,channels)))
test_model.summary()

## channels_inside = 10, channels input = 40
channels = 40
input_channels = 40
print("input_channels :", input_channels, "\nchannels :", channels)
test_tensors = tf.ones((nb_input,shape_1,shape_2,input_channels))
test_model = K.models.Sequential()
# test_pool = PoolBN("max", 3, 3, 1, "same")
test_model.add(PoolBN("max", input_channels, channels, 1, "same"))
test_model.add(PoolBN("max", channels, channels, 2, "same"))
test_model.add(FactorizedReduce(channels,channels))
# test_model.add(drop_path(0.3))
# test_model.add(SepConv(3,3,3,1,'valid'))

# test_model.add(Layer()) # this is apparently a identity layer
test_model.compile(loss = "mse")
test_model.fit(test_tensors, tf.ones((nb_input,shape_1//4,shape_2//4,channels)))
test_model.summary()


input_channels : 80 
channels : 20
1/1 [==============================] - 1s 1s/step - loss: 1.0000
Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 pool_bn_15 (PoolBN)         (None, 100, 100, 80)      320       
                                                                 
 pool_bn_16 (PoolBN)         (None, 50, 50, 80)        320       
                                                                 
 factorized_reduce_8 (Factor  (None, 25, 25, 20)       14480     
 izedReduce)                                                     
                                                                 
Total params: 15,120
Trainable params: 14,760
Non-trainable params: 360
_________________________________________________________________
input_channels : 40 
channels : 40
1/1 [==============================] - 1s 994ms/step - loss: 1.0000
Model: "sequential_14"
___________________

In [ ]:
import tensorflow as tf
nb_input = 10
shape_1 = 100
shape_2 = 100
## channels_inside = 10, channels input = 40
channels = 20
input_channels = 80
print("input_channels :", input_channels, "\nchannels :", channels)
test_tensors = tf.ones((nb_input,shape_1,shape_2,input_channels))
test_model = K.models.Sequential()
# test_pool = PoolBN("max", 3, 3, 1, "same")
test_model.add(DilConv(channels, channels, 1, 1, "same",1))
test_model.add(PoolBN("max", channels, channels, 2, "same"))
test_model.add(FactorizedReduce(channels,channels))
# test_model.add(drop_path(0.3))
# test_model.add(SepConv(3,3,3,1,'valid'))

# test_model.add(Layer()) # this is apparently a identity layer
test_model.compile(loss = "mse")
test_model.fit(test_tensors, tf.ones((nb_input,shape_1//4,shape_2//4,channels)))
test_model.summary()

## channels_inside = 10, channels input = 40
channels = 40
input_channels = 40
print("input_channels :", input_channels, "\nchannels :", channels)
test_tensors = tf.ones((nb_input,shape_1,shape_2,input_channels))
test_model = K.models.Sequential()
# test_pool = PoolBN("max", 3, 3, 1, "same")
test_model.add(DilConv(channels, channels, 1, 1, "same",1))
test_model.add(PoolBN("max", channels, channels, 2, "same"))
test_model.add(FactorizedReduce(channels,channels))
# test_model.add(drop_path(0.3))
# test_model.add(SepConv(3,3,3,1,'valid'))

# test_model.add(Layer()) # this is apparently a identity layer
test_model.compile(loss = "mse")
test_model.fit(test_tensors, tf.ones((nb_input,shape_1//4,shape_2//4,channels)))
test_model.summary()


input_channels : 80 
channels : 20
1/1 [==============================] - 2s 2s/step - loss: 1.0000
Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dil_conv_1 (DilConv)        (None, 100, 100, 20)      2120      
                                                                 
 pool_bn_13 (PoolBN)         (None, 50, 50, 20)        80        
                                                                 
 factorized_reduce_6 (Factor  (None, 25, 25, 20)       3590      
 izedReduce)                                                     
                                                                 
Total params: 5,790
Trainable params: 5,670
Non-trainable params: 120
_________________________________________________________________
input_channels : 40 
channels : 40
1/1 [==============================] - 1s 1s/step - loss: 1.0000
Model: "sequential_11"
_________________________

## Convolutions

Alternatives to convolutions to use in the model (based on dilation)

### Dilated convolution

In [ ]:
class DilConv(tf.keras.layers.Layer):
  """
  This class is the dilated depthwise separable convolution class.
  """
  def __init__(self, C_in, C_out, kernel_size, stride, padding, dilation, affine=True):
    """
    Description
    ---------------
    This function initializes an instance of the DilConv class.
    Input(s)
    ---------------
    C_in: int
    C_out: int
    kernel_size: int
    stride: int
    padding: int
    dilation: int
    affine: boolean
    Output(s)
    ---------------
    No output
    """
    super().__init__()
    self.c_in = C_in
    self.c_out = C_out
    self.k_size = kernel_size
    self.stride = stride
    self.padding = padding
    self.dilation = dilation
    self.affine = affine

    self.net = tf.keras.Sequential()
    self.net.add(tf.keras.layers.ReLU())
    self.net.add(tf.keras.layers.Conv2D(C_in, kernel_size, strides=stride, dilation_rate=dilation, padding="same",  data_format="channels_last" ) )
    self.net.add(tf.keras.layers.Conv2D(C_out, kernel_size, strides=(1, 1), padding=padding, data_format="channels_last"))
    self.net.add(tf.keras.layers.BatchNormalization() )
  
  def get_config(self):
        config = super().get_config()
        config.update({
            "C_in": self.c_in,
            "C_out": self.c_out,
            "kernel_size": self.k_size,
            "stride": self.stride,
            "padding": self.padding,
            "dilation": self.dilation,
            "affine": self.affine,
        })
        return config

  def call(self, x):
    """
    Description
    ---------------
    This function applies the instance of the DilConv class to the data.
    Input(s)
    ---------------
    x: tf.Tensor
    Output(s)
    ---------------
    output: tf.Tensor
    """
    # print("Dilconv",x)
    return self.net(x)

### Separable convolution

In [ ]:
class SepConv(tf.keras.layers.Layer):
  """
  Depthwise separable conv.
  DilConv(dilation=1) * 2.
  """
  def __init__(self, C_in, C_out, kernel_size, stride, padding, affine=True):
    """
    Description
    ---------------
    This function initializes an instance of the SepConv class.
    Input(s)
    ---------------
    C_in: int
    C_out: int
    kernel_size: int
    stride: int
    padding: int
    affine: boolean
    Output(s)
    ---------------
    No output
    """
    super().__init__()
    
    self.c_in = C_in
    self.c_out = C_out
    self.k_size = kernel_size
    self.stride = stride
    self.padding = padding
    self.affine = affine

    self.net = tf.keras.Sequential()
    self.net.add(DilConv(C_in, C_in, kernel_size, stride, padding, dilation=1,affine=affine))
    self.net.add(DilConv(C_in, C_out, kernel_size, 1, padding, dilation=1, affine=affine))

  def get_config(self):
        config = super().get_config()
        config.update({
            "C_in": self.c_in,
            "C_out": self.c_out,
            "kernel_size": self.k_size,
            "stride": self.stride,
            "padding": self.padding,
            "affine": self.affine,
        })
        return config
  
  def call(self, x):
    """
    Description
    ---------------
    This function applies the instance of the SepConv class to the data.
    Input(s)
    ---------------
    x: tf.Tensor
    Output(s)
    ---------------
    output: tf.Tensor
    """
    # print("sepconv",x)
    return self.net(x)

### Standard convolution

In [ ]:
class StdConv(tf.keras.layers.Layer):
  """
  Description
  ---------------
  This class is used to do an average convolution.
  It is composed of a relu followed by the convolution.
  Then a batchnormalisation is applied.
  The model used is a sequential keras model.
  """
  def __init__(self, C_in, C_out, kernel_size, stride, padding, affine=True):
    """
    Description
    ---------------
    Creates an instance of the class StdConv

    Input(s)
    ---------------
    C_in: int
    C_out: int
    kernel_size: int
    stride: int
    padding: string ("same" ou "valid")
    affine: bool

    Output(s)
    ---------------
    No output
    """
    # print("-----------entrée dans la fonction __init__ de la classe StdConv")
    super().__init__()
    
    self.c_in = C_in
    self.k_size = kernel_size
    self.stride = stride
    self.padding = padding
    self.affine = affine

    self.C_out = C_out
    self.relu = tf.keras.layers.ReLU()
    self.conv2d = tf.keras.layers.Conv2D(filters=C_out,
                                        kernel_size=kernel_size,
                                        strides=stride,
                                        padding=padding, 
                                        use_bias=False )
    self.batch_normalisation = tf.keras.layers.BatchNormalization(axis=-1)

  def get_config(self):
        config = super().get_config()
        config.update({
            "C_in": self.c_in,
            "C_out": self.C_out,
            "kernel_size": self.k_size,
            "stride": self.stride,
            "padding": self.padding,
            "affine": self.affine,
        })
        return config
    
  def call(self, x):
    """
    Description:
    ---------------
    Similar to the forward function in tensorflow, this function is used to train the AI.

    Input(s)
    ---------------
    x : tf.Tensor

    Output(s)
    ---------------
    output: tf.Tensor
    """
    # print("-----------------entrée dans la fonction call de la classe StdConv --------------")
    # print("x = {}".format(x))
    # x is a Tensor
    # print("x.get_shape() = {}".format(x.get_shape()))

    # print("x shape std conv", x.shape)
    out_relu = self.relu(x)
    # print("C_out", self.C_out)
    # print("out_relu shape std conv", out_relu.shape)
    # print("out_relu = {}".format(out_relu))
    # print("out_relu.get_shape()".format(out_relu.get_shape()))
    out_conv2d = self.conv2d(out_relu)
    # print("out_conv2d = {}".format(out_conv2d))
    # print("out_conv2d.get_shape() = {}".format(out_conv2d.get_shape()))
    out_batch_normalisation = self.batch_normalisation(out_conv2d)
    # print("out_batch_normalisation = {}".format(out_batch_normalisation))
    # print("out_batch_normalisation.get_shape() = {}".format(out_batch_normalisation.get_shape()))
    out = out_batch_normalisation
    
    # print("stdconv = {}".format(out))
    return out


In [ ]:
import tensorflow as tf
nb_input = 10
test_tensors = tf.ones((nb_input,120,120,9))
test_model = K.models.Sequential()
# test_pool = PoolBN("max", 3, 3, 1, "same")
test_model.add(SepConv(30,30,3,1,"same"))
test_model.add(DilConv(20,20,5,1,"same",1))
test_model.add(StdConv(10,10,5,1,"same"))
test_model.add(StdConv(20,20,5,1,"same"))
test_model.add(PoolBN("max", 3, 3, 1, "same"))
test_model.add(PoolBN("avg", 3, 3, 2, "same"))
test_model.add(FactorizedReduce(9,3))
test_model.add(drop_path(0.3))
# test_model.add(SepConv(3,3,3,1,'valid'))

# test_model.add(Layer()) # this is apparently a identity layer
test_model.compile(loss = "mse")
test_model.fit(test_tensors, tf.ones((nb_input,30,30,3)))
test_model.summary()

1/1 [==============================] - 6s 6s/step - loss: 1.6111
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sep_conv (SepConv)          (None, 120, 120, 30)      27090     
                                                                 
 dil_conv_2 (DilConv)        (None, 120, 120, 20)      25120     
                                                                 
 std_conv (StdConv)          (None, 120, 120, 10)      5040      
                                                                 
 std_conv_1 (StdConv)        (None, 120, 120, 20)      5080      
                                                                 
 pool_bn_6 (PoolBN)          (None, 120, 120, 20)      80        
                                                                 
 pool_bn_7 (PoolBN)          (None, 60, 60, 20)        80        
                                                       

# Model creation



## Importing the right plans

In [ ]:
input_dict_1 = {
    'normal_n2_p0': 'maxpool', 
    'normal_n2_p1': 'maxpool', 
    'normal_n3_p0': 'maxpool', 
    'normal_n3_p1': 'maxpool', 
    'normal_n3_p2': 'maxpool', 
    'normal_n4_p0': 'maxpool', 
    'normal_n4_p1': 'maxpool', 
    'normal_n4_p2': 'maxpool', 
    'normal_n4_p3': 'maxpool', 
    'normal_n5_p0': 'maxpool', 
    'normal_n5_p1': 'maxpool', 
    'normal_n5_p2': 'maxpool', 
    'normal_n5_p3': 'maxpool', 
    'normal_n5_p4': 'maxpool',
    'reduce_n2_p0': 'maxpool', 
    'reduce_n2_p1': 'maxpool', 
    'reduce_n3_p0': 'maxpool', 
    'reduce_n3_p1': 'sepconv5x5', 
    'reduce_n3_p2': 'maxpool', 
    'reduce_n4_p0': 'maxpool', 
    'reduce_n4_p1': 'maxpool', 
    'reduce_n4_p2': 'dilconv5x5', 
    'reduce_n4_p3': 'maxpool', 
    'reduce_n5_p0': 'maxpool', 
    'reduce_n5_p1': 'sepconv5x5', 
    'reduce_n5_p2': 'maxpool', 
    'reduce_n5_p3': 'dilconv5x5', 
    'reduce_n5_p4': 'maxpool', 
    'normal_n2_switch': [1, 0], 
    'normal_n3_switch': [2, 1], 
    'normal_n4_switch': [3, 2], 
    'normal_n5_switch': [2, 4], 
    'reduce_n2_switch': [1, 0], 
    'reduce_n3_switch': [2, 1], 
    'reduce_n4_switch': [3, 2], 
    'reduce_n5_switch': [3, 4] 
}
input_dict_norm_2 = {"2": {"0": "sepconv3x3", "1": "sepconv5x5"}, "3": {"1": "sepconv5x5", "0": "maxpool"}, "4": {"0": "avgpool", "2": "dilconv5x5"}, "5": {"1": "sepconv3x3", "0": "sepconv3x3"}}
input_dict_redu_2 = {"2": {"1": "maxpool", "0": "maxpool"}, "3": {"2": "maxpool", "1": "maxpool"}, "4": {"3": "maxpool", "2": "maxpool"}, "5": {"3": "maxpool", "2": "maxpool"}}
input_dict_norm_1 = {2: {1: 'maxpool', 0: 'maxpool'}, 3: {2: 'maxpool', 1: 'maxpool'}, 4: {3: 'maxpool', 2: 'maxpool'}, 5: {2: 'maxpool', 4: 'maxpool'}}
input_dict_redu_1 = {2: {1: 'maxpool', 0: 'maxpool'}, 3: {2: 'maxpool', 1: 'sepconv5x5'}, 4: {3: 'maxpool', 2: 'dilconv5x5'}, 5: {3: 'dilconv5x5', 4: 'maxpool'}}
input_dict_norm_test = {2: {0:"maxpool", 1: "avgpool"}, 3: {0:"skipconnect", 1: "avgpool"}, 4: {0:"sepconv3x3", 1: "sepconv5x5"}, 5: {0:"dilconv3x3", 1: "dilconv5x5"}}
input_dict_redu_test = {2: {0:"maxpool", 1: "avgpool"}, 3: {0:"skipconnect", 1: "avgpool"}, 4: {0:"sepconv3x3", 1: "sepconv5x5"}, 5: {0:"dilconv3x3", 1: "dilconv5x5"}}
print(input_dict_1["reduce_n5_switch"])
print(input_dict_norm_2["2"])
input_dict_norm_3 = {"2": {"1": "maxpool", "0": "maxpool"}, "3": {"2": "maxpool", "1": "maxpool"}, "4": {"2": "maxpool", "3": "maxpool"}, "5": {"4": "maxpool", "3": "maxpool"}}
input_dict_redu_3 = {"2": {"0": "maxpool", "1": "maxpool"}, "3": {"2": "maxpool", "0": "maxpool"}, "4": {"3": "maxpool", "2": "maxpool"}, "5": {"4": "dilconv5x5", "3": "maxpool"}}

[3, 4]
{'0': 'sepconv3x3', '1': 'sepconv5x5'}


## Classes

Creation of the Node, Cell and CNN classes (as layers) to use

In [ ]:
# for my custom version (cleaner and simpler + conversion already exists)

class Node(tf.keras.layers.Layer):
  def __init__(self, origin, element, channels, reducing):
    """
    Description
    ---------------
    Creates an instance of a node, the smallest base component of the CNN architecture.
    Input(s)
    ---------------
    origin: str (can be converted to int) : The id of the node being created
    element: dict : The plan of this specific node
    channels: int : Number of channels to use (must be the same for each node for compatibility reasons)
    reducing: boolean : Whether this node is used for reduction or not
    prev_reduction: boolean : Whether the previous node was a reduction node or not
    """
    super().__init__()
    self.origin = origin
    self.element = element
    self.channels = channels
    self.red = reducing
    self.models = dict()
    """
    self.preproc = tf.keras.layers.Layer()
    if prev_reduction:
      self.preproc = FactorizedReduce(channels, channels)
    """
    for i in element:
      stride = 2 if reducing and int(i) <=1 else 1
      dict_elements = {
          "maxpool":PoolBN("max", channels, 3, stride, "same"),
          "avgpool":PoolBN("avg", channels, 3, stride, "same"),
          "skipconnect":tf.keras.layers.Layer() if stride == 1 else FactorizedReduce(channels, channels),
          "sepconv3x3":SepConv(channels, channels, 3, stride, "same"),
          "sepconv5x5":SepConv(channels, channels, 5, stride, "same"),
          "dilconv3x3":DilConv(channels, channels, 3, stride, "same", 1),
          "dilconv5x5":DilConv(channels, channels, 5, stride, "same", 1) # Impossible to have a dilation greater than 1 if stride is not 1
      }
      self.models[int(i)] = dict_elements[element[i]]

  def get_config(self):
        config = super().get_config()
        config.update({
            "origin": self.origin,
            "reduction": self.red,
            "element": self.element,
            "channels": self.channels,
        })
        return config

  def call(self, x):
    """
    Description
    ---------------
    Function called when using the Node layer.
    Input(s)
    ---------------
    x: tf.Tensor
    Output(s)
    ---------------
    out: tf.Tensor
    """
    outs = []
    inputs = x
    i=0
    for key in self.models.keys():
      outs.append(self.models[key](inputs[key]))
      i+=1
    out = tf.keras.layers.Add()(outs)
    return out 

class Cell(tf.keras.layers.Layer):
  """
  Description
  ---------------
  Creates and returns a cell, according to a plan given as an input.
  Input(s)
  ---------------
  plan : dict containing the plan of the cell
  reduction : Model of the previous cell
  prev_reduction : Model of the cell before the previous cell
  Output(s)
  ---------------
  output_cell : Model of the normal cell
  """
  def __init__(self, plan, channels, reduction, prev_reduction):
    super().__init__()
    # Initiate the first two inputs => to add in the call
    # inputs = [prev_prev_cell, prev_cell]
    self.plan = plan
    self.chan = channels
    self.red = reduction
    self.prev_red = prev_reduction
    self.nodes = dict()
    if prev_reduction:
      self.preproc0 = FactorizedReduce(channels, channels)
    else:
      self.preproc0 = StdConv(channels, channels, 1, 1, "same")
    self.preproc1 = StdConv(channels, channels, 1, 1, "same")
    self.prev_reduction = prev_reduction
    for origin in plan.keys():
      # print(element)
      # print(origin)
      node = Node(origin, plan[origin], channels, reduction)
      self.nodes[int(origin)] = node

  def get_config(self):
        config = super().get_config()
        config.update({
            "plan": self.plan,
            "channels": self.chan,
            "reduction": self.red,
            "prev_reduction": self.prev_red,
        })
        return config

  def call(self, x):
    # inputs = tf.keras.Input(x)
    output = [self.preproc0(x[0]),self.preproc1(x[1])]
    for i in self.nodes:
      # print("node_nb",i)
      # print("self.nodes",self.nodes)
      output.append(self.nodes[i](output))
    # print("cell output", output[2:])
    added = tf.keras.layers.Concatenate(axis = -1)(output[2:])
    # print("added", added)
    # model = tf.keras.Model(inputs = x, outputs = output)
    return added

class CNN(tf.keras.layers.Layer):
  def __init__(self, channels, n_classes, n_layers, normal_plan, reduction_plan, stem_multiplier=3):
    """
    Description
    ---------------
    Creates an instance of a CNN layer.
    Input(s)
    ---------------
    channels: int : Number of channels in the internal outputs, must be equal to the number of channels of the input
    n_classes: int : number of classes we want to final output to have through a softmax
    n_layers: int : number of cells used in the model
    normal_plan: dict : plan to construct normal cells
    reduction_plan: dict: plan to construct reduction cells
    stem_multiplier: int : multiplier for the beginning
    """
    super().__init__()
    self.channels = channels
    self.n_classes = n_classes
    self.n_layers = n_layers
    self.normal_plan = normal_plan
    self.red_plan = reduction_plan
    self.stem_mul = stem_multiplier
    # Start : First layers
    cur_chan = stem_multiplier * channels
    self.stem = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(cur_chan, 3, strides=(1, 1), padding="same", data_format="channels_last"),
        tf.keras.layers.BatchNormalization(axis = -1)]
    )
    cur_chan = channels
    reduction_prev , reduction = False, False
    self.cells = []
    for i in range(n_layers):
      reduction_prev, reduction = reduction, False
      if i in [n_layers // 3, 2 * n_layers // 3]:
        cur_chan *= 2
        reduction=True
      
      # print("i",i,"red", reduction, )
      if reduction:
        cell = Cell(reduction_plan, cur_chan, reduction = True, prev_reduction = reduction_prev)
      else:
        cell = Cell(normal_plan, cur_chan, reduction = False, prev_reduction = reduction_prev)
      self.cells.append(cell)
    self.gap = tf.keras.layers.GlobalAveragePooling2D()
    self.linear = tf.keras.layers.Dense(n_classes, activation = "softmax")
  
  def get_config(self):
        config = super().get_config()
        config.update({
            "n_classes": self.n_classes,
            "n_layers": self.n_layers,
            "stem_multiplier": self.stem_mul,
            "channels": self.channels,
            "normal_plan": self.normal_plan,
            "reduction_plan": self.red_plan,
        })
        return config
  
  def call(self, x):
    """
    Description
    ---------------
    Function used for forward propagation.
    Input(s)
    ---------------
    x: Tensor
    Output(s)
    ---------------
    out: Tensor
    """
    # inputs = tf.keras.Input(x)
    s0 = s1 = self.stem(x)
    out = s1
    prev_out = s0
    i=0
    for cell in self.cells:
      # print("new cell",i," ----------------------------------------------------------")
      prev_out,out = out,cell([prev_out,out])
      i+=1
    out = self.gap(out)
    # print('left all cells')
    out_flat = tf.keras.layers.Flatten()(out)
    # print('left flatten')
    out = self.linear(out_flat)
    # print("final stretch")
    return out

Cell(input_dict_norm_2, 3, False, False)




### Version with function over classes

In [ ]:
def Node_func(element, channels, reducing, input):
  outs = []
  inputs = input
  for key in element:
    stride = 2 if reducing and int(key)<2 else 1
    dict_element = {
      "maxpool":PoolBN("max", channels, 3, stride, "same"),
      "avgpool":PoolBN("avg", channels, 3, stride, "same"),
      "skipconnect":tf.keras.layers.Layer() if stride == 1 else FactorizedReduce(channels, channels),
      "sepconv3x3":SepConv(channels, channels, 3, stride, "same"),
      "sepconv5x5":SepConv(channels, channels, 5, stride, "same"),
      "dilconv3x3":DilConv(channels, channels, 3, stride, "same", 1),
      "dilconv5x5":DilConv(channels, channels, 5, stride, "same", 1) # Impossible to have a dilation greater than 1 if stride is not 1
    }
    outs.append(dict_element[element[key]](inputs[int(key)]))
  # print('outs', outs)
  added = tf.keras.layers.Add()(outs)
  return added

def Cell_func(plan, channels, reduction, prev_reduction, x0, x1):
  preproc_1 = StdConv(channels, channels, 1, 1, "same")
  preproc_0 = StdConv(channels, channels, 1, 1, "same")
  if prev_reduction:
    preproc_0 = FactorizedReduce(channels, channels)  
  output = [preproc_0(x0), preproc_1(x1)]
  for key in plan.keys():
    output.append(Node_func(plan[key], channels, reduction, output))
  out = tf.keras.layers.Concatenate(axis = -1)(output[2:])
  return out

def CNN_creation(channels, n_classes, n_layers, normal_plan, reduction_plan, input_shape, stem_multiplier=3):
  in_x = tf.keras.Input(input_shape[1:])
  stem = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(channels*stem_multiplier, 3, strides=(1, 1), padding="same", data_format="channels_last"),
        tf.keras.layers.BatchNormalization(axis = -1)]
    )
  out_0 = out_1 = stem(in_x)
  red, prev_red = False, False
  cur_chans = channels
  for i in range(n_layers):
    prev_red = red
    red = i==n_layers//3 or i==2*n_layers//3
    plan = normal_plan
    if red :
      plan = reduction_plan
      cur_chans*=2
    out_0, out_1 = out_1, Cell_func(plan, cur_chans, red, prev_red, out_0, out_1)
  out = tf.keras.layers.GlobalAveragePooling2D()(out_1)
  out = tf.keras.layers.Flatten()(out)
  out = tf.keras.layers.Dense(n_classes, activation = "softmax")(out)
  return tf.keras.Model(inputs = in_x, outputs = out)


### Example use of the model

> The usage of the input_dict_test is to ensure all methods are working correctly



In [ ]:
"""import tensorflow as tf
nb_input = 10
channels = 16
output_size = 10
layer_amount = 6
test_tensors = tf.ones((nb_input,100,100,3))
test_model = tf.keras.models.Sequential()
test_model.add(CNN(channels, output_size, layer_amount, input_dict_norm_test, input_dict_redu_test,1))
# test_model.summary()
optimizer = tf.keras.optimizers.SGD()
loss = tf.keras.losses.categorical_crossentropy
test_model.compile(loss = loss, optimizer = optimizer)
test_model.fit(test_tensors, tf.ones((nb_input,output_size)))
test_model.summary()"""
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

log_dir_2 = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + " (1)"
tensorboard_callback_2 = tf.keras.callbacks.TensorBoard(log_dir=log_dir_2, histogram_freq=1)


nb_input = 10
channels = 1
output_size = 10
layer_amount = 5
test_tensors = tf.ones((nb_input,100,100,3))
optimizer = tf.keras.optimizers.SGD()
loss = tf.keras.losses.categorical_crossentropy

print("nb_channels", channels, "functions")
test_model_2 = CNN_creation(channels, output_size, layer_amount, input_dict_norm_test, input_dict_redu_test, test_tensors.shape)
test_model_2.compile(loss = loss, optimizer = optimizer)
test_model_2.fit(test_tensors, tf.ones((nb_input,output_size)), 
          callbacks=[tensorboard_callback_2])
test_model_2.summary()

print("nb_channels", channels, "class")
test_model = tf.keras.models.Sequential()
test_model.add(CNN(channels, output_size, layer_amount, input_dict_norm_test, input_dict_redu_test))
test_model.compile(loss = loss, optimizer = optimizer)
test_model.fit(test_tensors, tf.ones((nb_input,output_size)), 
          callbacks=[tensorboard_callback])
test_model.summary()



nb_channels 1 functions


KeyboardInterrupt: ignored

## Importing the cifar 10 data

In [ ]:
data = tf.keras.datasets.cifar10.load_data()

In [ ]:
train_data, test_data = data

In [ ]:
print(train_data[0].shape)
print(train_data[1].shape)
print(test_data[0].shape)
print(test_data[1].shape)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


## Creation of a model using cifar 10 and the plans

In [ ]:
import tensorflow as tf
import time

# nb_input = 10
batch_size = 64
channels = 4
output_size = 10
n_epochs = 5
layer_amount = 5
# Dataset creation
train_tensors = tf.cast(tf.convert_to_tensor(train_data[0]),tf.float32)
# print(train_tensors)
result_train_tensor = tf.keras.utils.to_categorical(tf.convert_to_tensor(train_data[1]))



import datetime
log_dir_3 = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + " new factorized_reduce class " + str(channels) 
tensorboard_callback_3 = tf.keras.callbacks.TensorBoard(log_dir=log_dir_3, histogram_freq=1, profile_batch = "1, 782")

log_dir_4 = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + " new factorized_reduce function " + str(channels)
tensorboard_callback_4 = tf.keras.callbacks.TensorBoard(log_dir=log_dir_4, histogram_freq=1, profile_batch = "2347, 3128")



test_tensors = tf.cast(tf.convert_to_tensor(test_data[0]),tf.float32)
result_test_tensor = tf.keras.utils.to_categorical(tf.convert_to_tensor(test_data[1]))
optimizer = tf.keras.optimizers.SGD()
loss = tf.keras.losses.categorical_crossentropy

start_time = time.time()
test_model = tf.keras.models.Sequential()
test_model.add(CNN(channels, output_size, layer_amount, input_dict_norm_3, input_dict_redu_3))
test_model.compile(loss = loss, optimizer = optimizer, metrics= ['accuracy'])
test_model.fit(train_tensors, result_train_tensor, epochs = n_epochs, batch_size = batch_size, 
          callbacks=[tensorboard_callback_3])
test_model.summary()
end_time = time.time()-start_time
print("Time taken class :", end_time//60, "m", end_time%60, "s")

channels = 10
start_time = time.time()
test_model_2 = CNN_creation(channels, output_size, layer_amount, input_dict_norm_3, input_dict_redu_3, train_tensors.shape)
test_model_2.compile(loss = loss, optimizer = optimizer, metrics= ['accuracy'])
test_model_2.fit(train_tensors, result_train_tensor, epochs = n_epochs, batch_size = batch_size, 
          callbacks=[tensorboard_callback_4])
test_model_2.summary()
end_time = time.time()-start_time
print("Time taken function :", end_time//60, "m", end_time%60, "s")

Epoch 1/5
782/782 [==============================] - 98s 118ms/step - loss: 1.8029 - accuracy: 0.3454
Epoch 2/5
782/782 [==============================] - 25s 32ms/step - loss: 1.5566 - accuracy: 0.4267
Epoch 3/5
782/782 [==============================] - 24s 31ms/step - loss: 1.4715 - accuracy: 0.4583
Epoch 4/5
782/782 [==============================] - 24s 31ms/step - loss: 1.4039 - accuracy: 0.4875
Epoch 5/5
782/782 [==============================] - 24s 31ms/step - loss: 1.3531 - accuracy: 0.5063
Model: "sequential_3291"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cnn_2 (CNN)                 (None, 10)                27546     
                                                                 
Total params: 27,546
Trainable params: 26,482
Non-trainable params: 1,064
_________________________________________________________________
Time taken class : 3.0 m 17.97048282623291 s
Epoch 1/5
782/782 [=

### Evaluation of the model

In [ ]:
test_model.evaluate(test_tensors, result_test_tensor)